In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import math
import os

# Custom quantization function with scale only (zero point is always 0)
class QuantizeFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, num_bits, scale, signed=True):
        if signed:
            qmin = -2 ** (num_bits - 1)
            qmax = 2 ** (num_bits - 1) - 1
        else:
            qmin = 0
            qmax = 2 ** num_bits - 1

        # Quantize with zero point = 0
        input_q = torch.round(input / scale).clamp(qmin, qmax)
        ctx.save_for_backward(scale)
        return input_q

    @staticmethod
    def backward(ctx, grad_output):
        # Straight-Through Estimator
        scale, = ctx.saved_tensors
        grad_input = grad_output.clone() / scale
        return grad_input, None, None, None

# Function to compute scale only (zero point is always 0)
def compute_scale(tensor, num_bits=8, signed=True):
    if signed:
        qmin = -2 ** (num_bits - 1)
        qmax = 2 ** (num_bits - 1) - 1
    else:
        qmin = 0
        qmax = 2 ** num_bits - 1

    max_val = (tensor.abs()).max()

    scale = -max_val/ float(qmax)
    return scale

# Custom quantized linear layer
class QuantLinear(nn.Module):
    def __init__(self, in_features, out_features):
        super(QuantLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        # Initialize weights and biases
        self.weight = nn.Parameter(torch.Tensor(out_features, in_features))
        self.bias = nn.Parameter(torch.Tensor(out_features))
        # Initialize parameters
        self.weight_q = None
        self.bias_q = None
        self.weight_scale = None
        self.bias_scale = None
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.weight)
        bound = 1 / math.sqrt(fan_in)
        nn.init.uniform_(self.bias, -bound, bound)

    def quantize_weights(self):
        # Compute scale for weights
        self.weight_scale = compute_scale(self.weight, num_bits=4, signed=True)
        self.weight_q = QuantizeFunction.apply(self.weight, 4, self.weight_scale, True).detach()

        # Compute scale for biases
        if self.bias is not None:
            self.bias_scale = compute_scale(self.bias, num_bits=4, signed=True)
            self.bias_q = QuantizeFunction.apply(self.bias, 4, self.bias_scale, True).detach()
        else:
            self.bias_q = None

    def forward(self, input):
        # Quantize weights and biases
        self.quantize_weights()

        # Quantize input
        input_scale = compute_scale(input, num_bits=16, signed=True)
        input_q = QuantizeFunction.apply(input, 16, input_scale, True)

        # Save input scale (if needed for export)
        self.input_scale = input_scale

        # Compute output
        output_q = torch.matmul(input_q, self.weight_q.t())
        if self.bias_q is not None:
            output_q += self.bias_q.unsqueeze(0).expand_as(output_q)

        return output_q

# Quantized model
class QuantComplexMLP(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, hidden_size4, hidden_size5, hidden_size6, hidden_size7, hidden_size8, output_size):
        super(QuantComplexMLP, self).__init__()
        # Define layers
        self.fc1 = QuantLinear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = QuantLinear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = QuantLinear(hidden_size2, hidden_size3)
        self.relu3 = nn.ReLU()
        self.fc4 = QuantLinear(hidden_size3, hidden_size4)
        self.relu4 = nn.ReLU()
        self.fc5 = QuantLinear(hidden_size4, hidden_size5)
        self.relu5 = nn.ReLU()
        self.fc6 = QuantLinear(hidden_size5, hidden_size6)
        self.relu6 = nn.ReLU()
        self.fc7 = QuantLinear(hidden_size6, hidden_size7)
        self.relu7 = nn.ReLU()
        self.fc8 = QuantLinear(hidden_size7, hidden_size8)
        self.relu8 = nn.ReLU()
        self.fc9 = QuantLinear(hidden_size8, output_size)

    def forward(self, x):
        x_scale = compute_scale(x, num_bits=16, signed=True)
        x = QuantizeFunction.apply(x, 16, x_scale, True)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.fc4(x)
        x = self.relu4(x)
        x = self.fc5(x)
        x = self.relu5(x)
        x = self.fc6(x)
        x = self.relu6(x)
        x = self.fc7(x)
        x = self.relu7(x)
        x = self.fc8(x)
        x = self.relu8(x)
        x = self.fc9(x)
        return x

# Device Configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load Data
df = pd.read_csv('dataset1.csv')

input_size = 27   # Adjust based on your actual data
output_size = 6   # Adjust based on your actual data

train_data_columns = df.columns[:27]
train_label_columns = df.columns[27:]

train_data = torch.tensor(df[train_data_columns].values, dtype=torch.float32)
train_labels = torch.tensor(df[train_label_columns].values, dtype=torch.float32)

# Preprocess labels if needed
train_labels[:99] = train_labels[:99] * 99  # Adjust indices as needed

train_dataset = TensorDataset(train_data, train_labels)
train_loader = DataLoader(train_dataset, batch_size=1000, shuffle=True)

# Initialize the model
model = QuantComplexMLP(input_size, 512, 1024, 1024, 1024, 1024, 512, 512, 256, output_size).to(device)

# Define Loss and Optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.000008)

# Training Loop
num_epochs = 3000  # Adjust as needed
for epoch in range(num_epochs):
    model.train()
    for batch_data, batch_labels in train_loader:
        batch_data = batch_data.to(device)
        batch_labels = batch_labels.to(device)

        optimizer.zero_grad()
        outputs = model(batch_data)
        loss = criterion(outputs.float(), batch_labels.float())
        loss.backward()
        optimizer.step()

    # Optional: Print loss every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Quantize the model after training for consistent parameters
def quantize_model(model):
    for module in model.modules():
        if isinstance(module, QuantLinear):
            module.quantize_weights()

quantize_model(model)

# Export quantized weights
import numpy as np  # Make sure to import numpy

def export_quantized_weights(model, output_dir='weights'):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    layer_idx = 0
    for name, module in model.named_modules():
        if isinstance(module, QuantLinear):
            layer_idx += 1

            # Use stored quantization parameters
            quantized_weights = module.weight_q.cpu().numpy().astype(int)
            scale_w = module.weight_scale.item()

            # Adjust signed 4-bit integers to unsigned for packing
            quantized_weights_uint = (quantized_weights + 8).astype(np.uint8)

            # Flatten and pack weights
            flat_weights = quantized_weights_uint.flatten()
            if flat_weights.size % 2 != 0:
                flat_weights = np.append(flat_weights, 0)
            packed_weights = (flat_weights[1::2] << 4) | flat_weights[::2]

            # Save packed weights to a COE file
            weight_file = os.path.join(output_dir, f'fc{layer_idx}_weight.coe')
            with open(weight_file, 'w') as f:
                f.write('memory_initialization_radix=16;\n')
                f.write('memory_initialization_vector=\n')
                hex_values = [f'{byte:02X}' for byte in packed_weights]
                f.write(',\n'.join(hex_values))
                f.write(';')
            print(f"Saved quantized weights for layer {layer_idx} to {weight_file}")

export_quantized_weights(model)


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn